# Explore the transactions table

In [0]:
dbutils.widgets.text(name="catalog", defaultValue='', label='field')
dbutils.widgets.text("schema", defaultValue='', label='field')

In [0]:
%sql
SELECT count(*) 
FROM ${catalog}.${schema}.transactions

In [0]:
%sql
DESCRIBE ${catalog}.${schema}.transactions

In [0]:
%sql
SELECT * FROM ${catalog}.${schema}.transactions LIMIT 10;

In [0]:
%sql
-- this won't work unless Auto Loader has been set up to record the file name on load
SELECT source_file, count(*) 
FROM ${catalog}.${schema}.transactions
GROUP BY source_file
ORDER BY source_file;

In [0]:
%sql
SELECT source_file, count(*) 
FROM ${catalog}.${schema}.fraud_reports
GROUP BY source_file
ORDER BY source_file;

# Simple Visualisations and Mixed SQL / Python in one Notebook

In [0]:
# Python vars populated from dbutils widgets
catalog = dbutils.widgets.get('catalog')
schema = dbutils.widgets.get('schema')

In [0]:
# load the data into a spark Data Frame
spark_df = spark.read.table(f"{catalog}.{schema}.transactions")
display(spark_df.groupby('countryOrig').sum('amount'))

Databricks visualization. Run in Databricks to view.

In [0]:
silver_df = spark.read.table(f"{catalog}.{schema}.silver_transactions")
display(silver_df)

Databricks visualization. Run in Databricks to view.

# Query the Gold Transactions View

In [0]:
%sql
select countryOrig , countryDest , type, value, is_fraud from (
    select countryOrig , countryDest , type, ROUND(sum(amount),2) as value, is_fraud 
    from ${catalog}.${schema}.gold_transactions
    where amount > 350000
    group by countryOrig , countryDest ,type, is_fraud
) 
order by countryOrig asc
;

In [0]:
%sql
select  is_fraud , count(*)
from ${catalog}.${schema}.gold_transactions
group by is_fraud